In [64]:
from pprint import pprint
from lxml import html
import requests
import re
from pymongo import MongoClient
import datetime

1)Написать приложение, которое собирает основные новости с сайтов news.mail.ru, lenta.ru, yandex.news
Для парсинга использовать xpath. Структура данных должна содержать:
название источника,
наименование новости,
ссылку на новость,
дата публикации

2)Сложить все новости в БД

In [73]:
client = MongoClient('127.0.0.1', 27017)
db = client['news']

In [75]:
now = datetime.datetime.now()
date = now.strftime("%d-%m-%Y")

## Новости с Яндэкс, по региону Москва.

In [58]:
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 YaBrowser/20.6.2.195 Yowser/2.5 Safari/537.36'}
link = 'https://yandex.ru/news/'
response = requests.get(link, headers=header)
dom = html.fromstring(response.text)
title = dom.xpath("//div[@aria-labelledby='Moscow']//tr//a[contains(@class,'link_theme_black')]/text()")
link = dom.xpath("//div[@aria-labelledby='Moscow']//tr//a[contains(@class,'link_theme_black')]/@href")
sourse = dom.xpath("//div[@aria-labelledby='Moscow']//tr//div[@class='story__date']/text()")

In [94]:
for ind,news in enumerate(title):
    db.news.insert_one({
        'title': news,
        'link':link[ind],
        'sourse':sourse[ind][:len(sourse[ind])-6],
        'date':date
    })

In [96]:
for i in db.news.find({}).limit(2):
    pprint(i)

{'_id': ObjectId('5f01d851c82aee057490f503'),
 'date': '05-07-2020',
 'link': '/news/story/Komnaty_prokhlady_otkrylis_v_stolichnykh_centrakh_socobsluzhivaniya--b55045f05518435f290bfed07beebf09?lr=213&lang=ru&stid=IHBYg5ZxMzFKelP1ceqw&persistent_id=104208121&rubric=Moscow&from=index',
 'sourse': 'РИАМО',
 'title': 'Комнаты прохлады открылись в столичных центрах соцобслуживания'}
{'_id': ObjectId('5f01d851c82aee057490f504'),
 'date': '05-07-2020',
 'link': '/news/story/V_baze_centra_zanyatosti_Moskvy_za_mesyac_poyavilis_pyat_tysyach_vakansij--3e8e35556d72171f9db21e2c908bc894?lr=213&lang=ru&stid=Oo6FKLt5Pe8Qd_nuFq8T&persistent_id=104718191&rubric=Moscow&from=index',
 'sourse': 'РИА Новости',
 'title': 'В базе центра занятости Москвы за месяц появились пять тысяч '
          'вакансий'}


## Новости с Лента, топ 7 новостей.

In [101]:
link = 'https://lenta.ru/'
response = requests.get(link, headers=header)
dom = html.fromstring(response.text)

In [102]:
title = dom.xpath("//section[@class = 'row b-top7-for-main js-top-seven']/div[contains(@class,'span4')]/div[contains(@class,'item')]//a[contains(@href,'news') and not(img)]/text()")
link = dom.xpath("//section[@class = 'row b-top7-for-main js-top-seven']/div[contains(@class,'span4')]/div[contains(@class,'item')]//a[contains(@href,'news') and not(img)]/@href")
date = dom.xpath("//section[@class = 'row b-top7-for-main js-top-seven']/div[contains(@class,'span4')]/div[contains(@class,'item')]//a[contains(@href,'news') and not(img)]/time/@datetime")

In [117]:
for ind,news in enumerate(title):
    db.news.insert_one({
        'title': news.replace('\xa0',' '),
        'link':link[ind],
        'sourse':'Lenta',
        'date':date[ind][-11:]
    })

In [118]:
for i in db.news.find({}).limit(2):
    pprint(i)

{'_id': ObjectId('5f01daaec82aee057490f53a'),
 'date': '5 июля 2020',
 'link': '/news/2020/07/05/vtoroi/',
 'sourse': 'Lenta',
 'title': 'В Австрии задержан второй подозреваемый в убийстве просителя '
          'убежища из России'}
{'_id': ObjectId('5f01daaec82aee057490f53b'),
 'date': '5 июля 2020',
 'link': '/news/2020/07/05/polgoda/',
 'sourse': 'Lenta',
 'title': 'Россиянин избил 10-летнего мальчика и полгода оставался '
          'безнаказанным'}


## Новости с мэил, политика.

In [119]:
link = 'https://news.mail.ru/'
response = requests.get(link, headers=header)
dom = html.fromstring(response.text)

In [120]:
links = dom.xpath("//div[@class='block block_separated_top rb_nat']/div/div/div/div[1]/div//a[contains(@href,'news')]/@href")

In [172]:
for link in links:
    resp_news = requests.get(link, headers=header)
    dom_news = html.fromstring(resp_news.text)
    date = dom_news.xpath("//span[@class='note__text breadcrumbs__text js-ago']/@datetime")
    title = dom_news.xpath("//h1[@class='hdr__inner']/text()")
    sourse = dom_news.xpath("//div[@class='breadcrumbs breadcrumbs_article js-ago-wrapper']/span[2]//span[@class='link__text']/text()")
    db.news.insert_one({
        'title': title[0],
        'link':link,
        'sourse':sourse[0],
        'date':date[0][:10]
    })

In [173]:
for i in db.news.find({}).limit(2):
    pprint(i)

{'_id': ObjectId('5f01df2dc82aee057490f578'),
 'date': '2020-07-05',
 'link': 'https://news.mail.ru/politics/42451941/',
 'sourse': 'BBC News Русская служба',
 'title': 'Рэпер Канье Уэст предложил себя на смену Трампу. Он это серьезно?'}
{'_id': ObjectId('5f01df2dc82aee057490f579'),
 'date': '2020-07-05',
 'link': 'https://news.mail.ru/politics/42449697/',
 'sourse': 'РБК',
 'title': 'Трамп пообещал США победу над радикалами вслед за коммунистами'}
